# AirREGI ヘルプデスク入電予測 - EDA & 特徴量エンジニアリング

## 概要
このノートブックでは、時系列データの探索的データ分析と特徴量エンジニアリングを行います。

### 設計方針
- **モジュール化**: 各特徴量グループをクラスで管理
- **テスト容易性**: 各特徴量を独立してテスト可能
- **データリーケージ防止**: 時系列データで未来の情報を使わない
- **可読性**: コードの意図を明確に

In [27]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from typing import List, Optional, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定（文字化け対策）
plt.rcParams['font.sans-serif'] = ['MS Gothic', 'Yu Gothic', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 表示設定
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

print("Setup complete!")

Setup complete!


## 1. データ読み込み

In [28]:
class DataLoader:
    """データ読み込みと前処理を行うクラス"""
    
    def __init__(self, input_dir: str = '../input'):
        self.input_dir = input_dir
        
    def load_all(self) -> Dict[str, pd.DataFrame]:
        """全データを読み込み、日付型に変換"""
        print("=" * 80)
        print("データ読み込み開始")
        print("=" * 80)
        
        # データ読み込み
        calender = pd.read_csv(f'{self.input_dir}/calender_data.csv')
        cm_data = pd.read_csv(f'{self.input_dir}/cm_data.csv')
        gt_service = pd.read_csv(f'{self.input_dir}/gt_service_name.csv')
        acc_get = pd.read_csv(f'{self.input_dir}/regi_acc_get_data_transform.csv')
        call_data = pd.read_csv(f'{self.input_dir}/regi_call_data_transform.csv')
        
        # 日付カラムをdatetime型に変換
        calender['cdr_date'] = pd.to_datetime(calender['cdr_date'])
        cm_data['cdr_date'] = pd.to_datetime(cm_data['cdr_date'])
        acc_get['cdr_date'] = pd.to_datetime(acc_get['cdr_date'])
        call_data['cdr_date'] = pd.to_datetime(call_data['cdr_date'])
        gt_service['week'] = pd.to_datetime(gt_service['week'])
        
        # データサイズを表示
        datasets = {
            'calender': calender,
            'cm_data': cm_data,
            'gt_service': gt_service,
            'acc_get': acc_get,
            'call_data': call_data
        }
        
        print("\n読み込み完了:")
        for name, df in datasets.items():
            print(f"  {name:15s}: {df.shape}")
        
        return datasets
    
    def merge_all(self, datasets: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        """全データを統合"""
        print("\n" + "=" * 80)
        print("データ統合開始")
        print("=" * 80)
        
        # メインデータ（入電数）を基準
        df = datasets['call_data'].copy()
        print(f"\nベースデータ: {df.shape}")
        
        # カレンダー情報をマージ
        df = df.merge(datasets['calender'], on='cdr_date', how='left')
        print(f"カレンダー統合後: {df.shape}")
        
        # CM情報をマージ
        df = df.merge(datasets['cm_data'], on='cdr_date', how='left')
        print(f"CM統合後: {df.shape}")
        
        # アカウント取得数をマージ
        df = df.merge(datasets['acc_get'], on='cdr_date', how='left')
        print(f"アカウント取得数統合後: {df.shape}")
        
        # Google Trendsを週次→日次に展開
        gt_daily = self._expand_weekly_to_daily(datasets['gt_service'])
        df = df.merge(gt_daily, on='cdr_date', how='left')
        print(f"Google Trends統合後: {df.shape}")
        
        # 日付でソート（時系列処理のため必須）
        df = df.sort_values('cdr_date').reset_index(drop=True)
        print("\n日付でソート完了（時系列処理のため）")
        
        # 欠損値確認
        print("\n欠損値の数（上位10）:")
        missing = df.isnull().sum().sort_values(ascending=False).head(10)
        for col, count in missing.items():
            if count > 0:
                print(f"  {col:30s}: {count:4d} ({count/len(df)*100:.1f}%)")
        
        return df
    
    @staticmethod
    def _expand_weekly_to_daily(gt_service: pd.DataFrame) -> pd.DataFrame:
        """週次データを日次に展開"""
        print("\nGoogle Trendsを週次→日次に展開中...")
        daily_records = []
        
        for _, row in gt_service.iterrows():
            week_start = row['week']
            for i in range(7):
                date = week_start + timedelta(days=i)
                daily_records.append({
                    'cdr_date': date,
                    'search_cnt': row['search_cnt']
                })
        
        return pd.DataFrame(daily_records)


# データ読み込み実行
loader = DataLoader()
datasets = loader.load_all()
df_raw = loader.merge_all(datasets)

print("\n" + "=" * 80)
print(f"統合データ: {df_raw.shape}")
print(f"期間: {df_raw['cdr_date'].min()} ~ {df_raw['cdr_date'].max()}")
print("=" * 80)


データ読み込み開始

読み込み完了:
  calender       : (670, 10)
  cm_data        : (762, 2)
  gt_service     : (109, 2)
  acc_get        : (701, 2)
  call_data      : (670, 2)

データ統合開始

ベースデータ: (670, 2)
カレンダー統合後: (670, 11)
CM統合後: (670, 12)
アカウント取得数統合後: (670, 13)

Google Trendsを週次→日次に展開中...
Google Trends統合後: (670, 14)

日付でソート完了（時系列処理のため）

欠損値の数（上位10）:
  holiday_name                  :  632 (94.3%)

統合データ: (670, 14)
期間: 2018-06-01 00:00:00 ~ 2020-03-31 00:00:00


## 2. 特徴量エンジニアリング

### 設計パターン
各特徴量グループを独立したクラスとして実装し、テストと保守を容易にします。

In [29]:
from abc import ABC, abstractmethod

class BaseFeatureEngineer(ABC):
    """特徴量エンジニアリングの基底クラス"""
    
    def __init__(self, name: str):
        self.name = name
        self.created_features: List[str] = []
    
    @abstractmethod
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """特徴量を作成（サブクラスで実装）"""
        pass
    
    def get_feature_names(self) -> List[str]:
        """作成された特徴量名のリストを取得"""
        return self.created_features
    
    def describe(self, df: pd.DataFrame) -> pd.DataFrame:
        """特徴量の統計情報を取得"""
        if not self.created_features:
            print(f"{self.name}: 特徴量が未作成です")
            return pd.DataFrame()
        return df[self.created_features].describe()


class TimeBasedFeatures(BaseFeatureEngineer):
    """日付から派生する基本的な時系列特徴量
    
    これらは未来の情報を使わないため、データリーケージの心配がありません。
    """
    
    def __init__(self):
        super().__init__("時系列基本特徴量")
    
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        # 年月日の特徴量
        df['year'] = df['cdr_date'].dt.year
        df['month'] = df['cdr_date'].dt.month
        df['day_of_month'] = df['cdr_date'].dt.day
        df['quarter'] = df['cdr_date'].dt.quarter
        df['day_of_year'] = df['cdr_date'].dt.dayofyear
        df['week_of_year'] = df['cdr_date'].dt.isocalendar().week
        
        # 経過日数
        df['days_from_start'] = (df['cdr_date'] - df['cdr_date'].min()).dt.days
        
        # 月初・月末フラグ
        df['is_month_start'] = (df['day_of_month'] <= 5).astype(int)
        df['is_month_end'] = (df['day_of_month'] >= 25).astype(int)
        
        # 週初・週末（既存のdowを利用）
        if 'dow' in df.columns:
            df['is_week_start'] = (df['dow'] == 1).astype(int)  # 月曜
            df['is_week_end'] = (df['dow'] == 5).astype(int)    # 金曜
        
        self.created_features = [
            'year', 'month', 'day_of_month', 'quarter', 'day_of_year',
            'week_of_year', 'days_from_start', 'is_month_start', 'is_month_end',
            'is_week_start', 'is_week_end'
        ]
        
        print(f"{self.name}: {len(self.created_features)}個の特徴量を作成")
        return df


class LagFeatures(BaseFeatureEngineer):
    """ラグ特徴量（過去のデータ）
    
    重要:
    - shift()を使って未来の情報が混入しないようにする
    - データは日付順にソート済みであることが前提
    """
    
    def __init__(self, target_col: str = 'call_num', lags: List[int] = [1, 2, 3, 5, 7, 14, 30]):
        super().__init__("ラグ特徴量")
        self.target_col = target_col
        self.lags = lags
    
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        if self.target_col not in df.columns:
            print(f"警告: {self.target_col}が見つかりません")
            return df
        
        for lag in self.lags:
            col_name = f'lag_{lag}'
            df[col_name] = df[self.target_col].shift(lag)
            self.created_features.append(col_name)
        
        print(f"{self.name}: {len(self.created_features)}個の特徴量を作成")
        print(f"  対象変数: {self.target_col}")
        print(f"  ラグ: {self.lags}")
        print(f"  注意: 最初の{max(self.lags)}日間はNaNになります")
        
        return df


class RollingFeatures(BaseFeatureEngineer):
    """移動統計量特徴量（移動平均、移動標準偏差など）
    
    重要:
    - rolling()の前にshift(1)を適用してデータリーケージを防止
    - 当日のデータが含まれないようにする
    """
    
    def __init__(self, target_col: str = 'call_num', windows: List[int] = [3, 7, 14, 30]):
        super().__init__("移動統計量特徴量")
        self.target_col = target_col
        self.windows = windows
    
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        if self.target_col not in df.columns:
            print(f"警告: {self.target_col}が見つかりません")
            return df
        
        for window in self.windows:
            # 移動平均（当日を含まない）
            ma_col = f'ma_{window}'
            df[ma_col] = df[self.target_col].shift(1).rolling(
                window=window, min_periods=1
            ).mean()
            self.created_features.append(ma_col)
            
            # 移動標準偏差（変動性を捉える）
            std_col = f'ma_std_{window}'
            df[std_col] = df[self.target_col].shift(1).rolling(
                window=window, min_periods=1
            ).std()
            self.created_features.append(std_col)
            
            # 移動最大値
            max_col = f'ma_max_{window}'
            df[max_col] = df[self.target_col].shift(1).rolling(
                window=window, min_periods=1
            ).max()
            self.created_features.append(max_col)
            
            # 移動最小値
            min_col = f'ma_min_{window}'
            df[min_col] = df[self.target_col].shift(1).rolling(
                window=window, min_periods=1
            ).min()
            self.created_features.append(min_col)
        
        print(f"{self.name}: {len(self.created_features)}個の特徴量を作成")
        print(f"  対象変数: {self.target_col}")
        print(f"  ウィンドウ: {self.windows}")
        print(f"  統計量: 平均, 標準偏差, 最大値, 最小値")
        
        return df


class DomainFeatures(BaseFeatureEngineer):
    """ドメイン知識に基づく特徴量
    
    - CM効果の累積
    - Google Trendsの平滑化
    - アカウント取得数の傾向
    - 曜日ごとの過去平均
    """
    
    def __init__(self):
        super().__init__("ドメイン特徴量")
    
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        # CM効果の累積（過去7日間のCM実施回数）
        if 'cm_flg' in df.columns:
            df['cm_7d_sum'] = df['cm_flg'].shift(1).rolling(window=7, min_periods=1).sum()
            df['cm_14d_sum'] = df['cm_flg'].shift(1).rolling(window=14, min_periods=1).sum()
            self.created_features.extend(['cm_7d_sum', 'cm_14d_sum'])
        
        # Google Trendsの移動平均（ノイズ除去）
        if 'search_cnt' in df.columns:
            df['gt_ma_7'] = df['search_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
            df['gt_ma_14'] = df['search_cnt'].shift(1).rolling(window=14, min_periods=1).mean()
            self.created_features.extend(['gt_ma_7', 'gt_ma_14'])
        
        # アカウント取得数の移動平均
        if 'acc_get_cnt' in df.columns:
            df['acc_ma_7'] = df['acc_get_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
            df['acc_ma_14'] = df['acc_get_cnt'].shift(1).rolling(window=14, min_periods=1).mean()
            self.created_features.extend(['acc_ma_7', 'acc_ma_14'])
        
        # 曜日ごとの過去平均（同じ曜日のパターンを捉える）
        if 'dow' in df.columns and 'call_num' in df.columns:
            df['dow_avg'] = np.nan
            for dow in df['dow'].unique():
                mask = df['dow'] == dow
                df.loc[mask, 'dow_avg'] = df.loc[mask, 'call_num'].shift(1).expanding().mean()
            self.created_features.append('dow_avg')
        
        print(f"{self.name}: {len(self.created_features)}個の特徴量を作成")
        return df


print("特徴量エンジニアリングクラス定義完了")
df.columns

特徴量エンジニアリングクラス定義完了


Index(['cdr_date', 'call_num'], dtype='object')

In [30]:
# セル4を実行済みなら、これで確認できる
print(type(datasets))  # <class 'dict'>
print(datasets.keys())  # dict_keys(['calender', 'cm_data', 'gt_service', 'acc_get', 'call_data'])

# calenderデータにアクセス
print(datasets['calender']['cdr_date'].dtype)  # datetime64[ns]
print(datasets['calender']['cdr_date'].head())

<class 'dict'>
dict_keys(['calender', 'cm_data', 'gt_service', 'acc_get', 'call_data'])
datetime64[ns]
0   2018-06-01
1   2018-06-02
2   2018-06-03
3   2018-06-04
4   2018-06-05
Name: cdr_date, dtype: datetime64[ns]


In [31]:
import os

# 保存先ディレクトリの作成
output_dir = '../output/datasets'
os.makedirs(output_dir, exist_ok=True)

# 各DataFrameをCSVとして保存
for name, df in datasets.items():
    filepath = f'{output_dir}/{name}.csv'
df.to_csv(filepath, index=False, encoding='utf-8')
print(f"保存完了: {filepath} ({df.shape})")

保存完了: ../output/datasets/call_data.csv ((670, 2))


In [32]:
"""
データ構造を確認するスクリプト
EDAノートブックのセル4を実行した後に実行してください
"""

import pandas as pd
from datetime import timedelta

# ==========================================
# サンプルデータで説明
# ==========================================

print("=" * 80)
print("データ構造の説明（サンプルデータ）")
print("=" * 80)

# 1. 個別データ（結合前）
print("\n" + "=" * 80)
print("1. 結合前の個別データ（datasets）")
print("=" * 80)

call_data = pd.DataFrame({
    'cdr_date': pd.to_datetime(['2018-06-01', '2018-06-02', '2018-06-03']),
    'call_num': [183, 0, 96]
})
print("\n[call_data] - 入電数（メインデータ）")
print(call_data)
print(f"shape: {call_data.shape} （{call_data.shape[0]}行 × {call_data.shape[1]}列）")
print(f"columns: {call_data.columns.tolist()}")

calender = pd.DataFrame({
    'cdr_date': pd.to_datetime(['2018-06-01', '2018-06-02', '2018-06-03']),
    'dow': [5, 6, 7],
    'dow_name': ['Friday', 'Saturday', 'Sunday'],
    'holiday_flag': [0, 0, 0]
})
print("\n[calender] - カレンダー情報")
print(calender)
print(f"shape: {calender.shape} （{calender.shape[0]}行 × {calender.shape[1]}列）")
print(f"columns: {calender.columns.tolist()}")

cm_data = pd.DataFrame({
    'cdr_date': pd.to_datetime(['2018-06-01', '2018-06-02', '2018-06-03']),
    'cm_flg': [0, 1, 0]
})
print("\n[cm_data] - CM実施フラグ")
print(cm_data)
print(f"shape: {cm_data.shape} （{cm_data.shape[0]}行 × {cm_data.shape[1]}列）")
print(f"columns: {cm_data.columns.tolist()}")

# 2. 結合処理
print("\n" + "=" * 80)
print("2. 結合処理（merge）")
print("=" * 80)

df = call_data.copy()
print(f"\nステップ1: call_dataをコピー")
print(f"  shape: {df.shape}")
print(f"  columns: {df.columns.tolist()}")

df = df.merge(calender, on='cdr_date', how='left')
print(f"\nステップ2: calenderを結合")
print(f"  shape: {df.shape}")
print(f"  columns: {df.columns.tolist()}")
print("  ↑ call_dataのカラム + calenderのカラム（cdr_date以外）")

df = df.merge(cm_data, on='cdr_date', how='left')
print(f"\nステップ3: cm_dataを結合")
print(f"  shape: {df.shape}")
print(f"  columns: {df.columns.tolist()}")
print("  ↑ さらにcm_dataのカラム（cdr_date以外）を追加")

# 3. 結合後のデータ
print("\n" + "=" * 80)
print("3. 結合後のデータ（df_raw）")
print("=" * 80)

print("\n[df_raw] - 全データが横に結合された1つのテーブル")
print(df)
print(f"\nshape: {df.shape} （{df.shape[0]}行 × {df.shape[1]}列）")
print(f"columns: {df.columns.tolist()}")

# 4. データの取り出し方
print("\n" + "=" * 80)
print("4. データの取り出し方")
print("=" * 80)

print("\n■ 1つの列を取り出す")
print("df['call_num']")
print(df['call_num'])

print("\n■ 複数の列を取り出す")
print("df[['cdr_date', 'call_num', 'dow']]")
print(df[['cdr_date', 'call_num', 'dow']])

print("\n■ 1行目のデータ")
print("df.iloc[0]")
print(df.iloc[0])

print("\n■ 特定の値")
print("df.loc[0, 'call_num']  # 1行目のcall_num")
print(df.loc[0, 'call_num'])

# 5. df.columnsの説明
print("\n" + "=" * 80)
print("5. df.columnsの説明")
print("=" * 80)

print(f"\ndf.columns = {df.columns}")
print(f"型: {type(df.columns)}")
print(f"データ型: {df.columns.dtype}")

print("\n■ 列名をリストとして取得")
print(f"df.columns.tolist() = {df.columns.tolist()}")

print("\n■ 列数")
print(f"len(df.columns) = {len(df.columns)}")

print("\n■ 列名を1つずつ表示")
for i, col in enumerate(df.columns):
    print(f"  {i}: {col}")

# 6. 実際のテーブル構造を視覚化
print("\n" + "=" * 80)
print("6. テーブル構造の視覚化")
print("=" * 80)

print("\n結合前:")
print("""
call_data          calender              cm_data
┌────────────┐    ┌─────────────────┐    ┌───────────┐
│ cdr_date   │    │ cdr_date        │    │ cdr_date  │
│ call_num   │    │ dow             │    │ cm_flg    │
└────────────┘    │ dow_name        │    └───────────┘
   2列            │ holiday_flag    │       2列
                  └─────────────────┘
                        4列
""")

print("\n結合後（横に並べる）:")
print("""
df_raw
┌───────────────────────────────────────────────────┐
│ cdr_date  call_num  dow  dow_name  holiday_flag  cm_flg │
└───────────────────────────────────────────────────┘
                    6列（全て横に並ぶ）
""")

print("\n行の構造:")
for idx, row in df.iterrows():
    print(f"\n{idx}行目:")
    print(f"  cdr_date     : {row['cdr_date']}")
    print(f"  call_num     : {row['call_num']}")
    print(f"  dow          : {row['dow']}")
    print(f"  dow_name     : {row['dow_name']}")
    print(f"  holiday_flag : {row['holiday_flag']}")
    print(f"  cm_flg       : {row['cm_flg']}")

print("\n" + "=" * 80)
print("説明完了")
print("=" * 80)


データ構造の説明（サンプルデータ）

1. 結合前の個別データ（datasets）

[call_data] - 入電数（メインデータ）
    cdr_date  call_num
0 2018-06-01       183
1 2018-06-02         0
2 2018-06-03        96
shape: (3, 2) （3行 × 2列）
columns: ['cdr_date', 'call_num']

[calender] - カレンダー情報
    cdr_date  dow  dow_name  holiday_flag
0 2018-06-01    5    Friday             0
1 2018-06-02    6  Saturday             0
2 2018-06-03    7    Sunday             0
shape: (3, 4) （3行 × 4列）
columns: ['cdr_date', 'dow', 'dow_name', 'holiday_flag']

[cm_data] - CM実施フラグ
    cdr_date  cm_flg
0 2018-06-01       0
1 2018-06-02       1
2 2018-06-03       0
shape: (3, 2) （3行 × 2列）
columns: ['cdr_date', 'cm_flg']

2. 結合処理（merge）

ステップ1: call_dataをコピー
  shape: (3, 2)
  columns: ['cdr_date', 'call_num']

ステップ2: calenderを結合
  shape: (3, 5)
  columns: ['cdr_date', 'call_num', 'dow', 'dow_name', 'holiday_flag']
  ↑ call_dataのカラム + calenderのカラム（cdr_date以外）

ステップ3: cm_dataを結合
  shape: (3, 6)
  columns: ['cdr_date', 'call_num', 'dow', 'dow_name', 'holiday_flag', '

In [ ]:
# ==============================================
# CSVファイル品質検証 (Data Quality Check)
# ==============================================
import pandas as pd
import numpy as np

print("=" * 80)
print("CSVファイル品質検証")
print("=" * 80)

# 全ファイル読み込み
files = {
    'call_data': '../input/regi_call_data_transform.csv',
    'acc_get': '../input/regi_acc_get_data_transform.csv',
    'calender': '../input/calender_data.csv',
    'cm_data': '../input/cm_data.csv',
    'gt_service': '../input/gt_service_name.csv'
}

datasets = {}
for name, path in files.items():
    datasets[name] = pd.read_csv(path)

# ==============================================
# 1. 基本情報
# ==============================================
print("\n【1. 基本情報】")
print("-" * 60)
for name, df in datasets.items():
    print(f"\n{name}:")
    print(f"  行数: {len(df)}, 列数: {len(df.columns)}")
    print(f"  カラム: {df.columns.tolist()}")

# ==============================================
# 2. 日付範囲の整合性チェック
# ==============================================
print("\n" + "=" * 80)
print("【2. 日付範囲の整合性チェック】")
print("-" * 60)

# 日付カラムを変換
datasets['call_data']['cdr_date'] = pd.to_datetime(datasets['call_data']['cdr_date'])
datasets['acc_get']['cdr_date'] = pd.to_datetime(datasets['acc_get']['cdr_date'])
datasets['calender']['cdr_date'] = pd.to_datetime(datasets['calender']['cdr_date'])
datasets['cm_data']['cdr_date'] = pd.to_datetime(datasets['cm_data']['cdr_date'])
datasets['gt_service']['week'] = pd.to_datetime(datasets['gt_service']['week'])

date_ranges = {}
for name in ['call_data', 'acc_get', 'calender', 'cm_data']:
    df = datasets[name]
    date_ranges[name] = (df['cdr_date'].min(), df['cdr_date'].max(), len(df))
    print(f"{name:15s}: {df['cdr_date'].min().date()} ~ {df['cdr_date'].max().date()} ({len(df)} rows)")

print(f"{'gt_service':15s}: {datasets['gt_service']['week'].min().date()} ~ {datasets['gt_service']['week'].max().date()} ({len(datasets['gt_service'])} weeks)")

# 日付の不一致を検出
print("\n⚠️  日付範囲の問題:")
call_dates = set(datasets['call_data']['cdr_date'])
cal_dates = set(datasets['calender']['cdr_date'])
acc_dates = set(datasets['acc_get']['cdr_date'])
cm_dates = set(datasets['cm_data']['cdr_date'])

# call_dataに存在しない日付
missing_in_call = cal_dates - call_dates
if missing_in_call:
    print(f"  - calenderにあるがcall_dataにない日付: {len(missing_in_call)}件")
    
missing_in_cal = call_dates - cal_dates
if missing_in_cal:
    print(f"  - call_dataにあるがcalenderにない日付: {len(missing_in_cal)}件")

# ==============================================
# 3. 欠損値チェック
# ==============================================
print("\n" + "=" * 80)
print("【3. 欠損値チェック】")
print("-" * 60)

for name, df in datasets.items():
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"\n{name}:")
        for col, count in missing.items():
            if count > 0:
                print(f"  {col}: {count} ({count/len(df)*100:.1f}%)")
    else:
        print(f"{name}: 欠損値なし ✓")

# ==============================================
# 4. データ型・値の妥当性チェック
# ==============================================
print("\n" + "=" * 80)
print("【4. データ型・値の妥当性チェック】")
print("-" * 60)

# call_num
call_df = datasets['call_data']
print(f"\ncall_data.call_num:")
print(f"  範囲: {call_df['call_num'].min()} ~ {call_df['call_num'].max()}")
print(f"  負の値: {(call_df['call_num'] < 0).sum()}件")
print(f"  ゼロの値: {(call_df['call_num'] == 0).sum()}件 (休業日?)")

# acc_get_cnt (正規化されている?)
acc_df = datasets['acc_get']
print(f"\nacc_get.acc_get_cnt:")
print(f"  範囲: {acc_df['acc_get_cnt'].min():.4f} ~ {acc_df['acc_get_cnt'].max():.4f}")
print(f"  平均: {acc_df['acc_get_cnt'].mean():.4f}")
print(f"  標準偏差: {acc_df['acc_get_cnt'].std():.4f}")
if acc_df['acc_get_cnt'].min() < -5 or acc_df['acc_get_cnt'].max() > 5:
    print("  ⚠️  正規化データとしては範囲が広い")
else:
    print("  ✓ 正規化されたデータと思われる (Zスコア変換?)")

# cm_flg
cm_df = datasets['cm_data']
print(f"\ncm_data.cm_flg:")
print(f"  ユニーク値: {cm_df['cm_flg'].unique()}")
print(f"  CM実施日: {cm_df['cm_flg'].sum()}日 / {len(cm_df)}日")

# calender
cal_df = datasets['calender']
print(f"\ncalender:")
print(f"  dow (曜日): {sorted(cal_df['dow'].unique())}")
print(f"  holiday_flag: {cal_df['holiday_flag'].unique()}")

# ==============================================
# 5. 重複チェック
# ==============================================
print("\n" + "=" * 80)
print("【5. 重複チェック】")
print("-" * 60)

for name, df in datasets.items():
    date_col = 'week' if name == 'gt_service' else 'cdr_date'
    duplicates = df[date_col].duplicated().sum()
    if duplicates > 0:
        print(f"{name}: ⚠️  {duplicates}件の重複日付")
    else:
        print(f"{name}: 重複なし ✓")

# ==============================================
# 6. 異常値・外れ値の疑い
# ==============================================
print("\n" + "=" * 80)
print("【6. 外れ値の検出】")
print("-" * 60)

# call_num (営業日のみ)
working_days = call_df[call_df['call_num'] > 0]['call_num']
Q1, Q3 = working_days.quantile([0.25, 0.75])
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR

outliers = working_days[working_days > upper_bound]
print(f"\ncall_num外れ値 (IQR法, 上限{upper_bound:.0f}超):")
print(f"  {len(outliers)}件検出")
if len(outliers) > 0:
    outlier_dates = call_df[call_df['call_num'] > upper_bound][['cdr_date', 'call_num']]
    outlier_dates = outlier_dates.sort_values('call_num', ascending=False)
    print(outlier_dates.head(10).to_string(index=False))

# ==============================================
# 総合判定
# ==============================================
print("\n" + "=" * 80)
print("【総合判定】")
print("=" * 80)

issues = []

# 日付範囲
if len(missing_in_call) > 0 or len(missing_in_cal) > 0:
    issues.append("日付範囲の不一致あり")

# acc_get_cntが正規化済み
issues.append("acc_get_cntは正規化済み（元の値ではない）")

# 外れ値
if len(outliers) > 0:
    issues.append(f"call_numに{len(outliers)}件の外れ値（2019年9月に集中）")

print("\n⚠️  注意点:")
for i, issue in enumerate(issues, 1):
    print(f"  {i}. {issue}")

print("\n✓ 信頼できる点:")
print("  - 重複日付なし")
print("  - 欠損値は最小限（holiday_nameのNAのみ）")
print("  - 日付形式は統一されている")
print("  - フラグ値は正常（0/1）")
